# 第7章  マージソート・数学の問題・線形計画法

 * 左上の「ファイル」を開いて、「ドライブにコピーを保存」をしてください。
 * 自分のGoogleドライブに、そのファイルが保存されます。ファイル名は「のコピー」が、最後（右端）についています。それを利用して学習・演習を進めて下さい。

# 7.6 線形計画法

次の問題を考える。

* 文化祭ですずらんとかすみ草を使って花束セットAと花束セットBを作って売ることにしました。

  * 花束セットAは1束につき すずらん3本、かすみ草5本を使い、売値は300円
  * 花束セットBは1束につき すずらん2本、かすみ草3本を使い、売値は190円

当日確保できた花は、すずらん90本、かすみ草140本でした。さて、この材料で作った花束セットAと花束セットBが全部売り切れると仮定すると、売上げを最大にするには、花束セットAと花束セットBをそれぞれ何束作っておけばいいでしょうか?

この問題では、花束セットAの個数を$x$、花束セットBの個数を$y$と置き、与えられた条件を数式で表し、さらに売上げを表す変数を$P$と置き、それを表計算ソフトを利用して解くという次のような手順をたどればよい。

* すずらんについて $3x+2y\le 90$……（ア）
* かすみ草について $5x+3y\le 140$……（イ）
* $ P=300x+190y $……（ウ）

シミュレーションによって$P$の最大値を探す。

数学的に見れば、数式（ア）（イ）（ウ）は、線形計画法に典型的な問題であるが、$x, y$の値は非負整数上を動くため、$P$の最大値問題を解くことは数学的には容易ではない。一方、この問題を表計算ソフトやプログラミングで分析することができれば、数学の解法よりもはるかに簡単に正解を求めることができる。

* 14行目： 花束セットAを0個と仮定し、
* 17行目から18行目：花束セットAの個数を一つずづ増やしながら、作れる花束セットA、Bの個数を求め、売上を計算する。
* 21行目から22行目：今までの売上最大値よりも大きな値が観測されたら、それを覚える。
* 23行目から25行目：花束セットAを一つ増やすと、花束セットBの材料となるすずらんとかすみ草の数が減る。
* 27行目以降：覚えた結果を表示する。


【プログラム7601】

In [ ]:
import matplotlib.pyplot as plt

# 状況の設定
suzuran = 90
kasumi = 140
a_suzuran, a_kasumi, a_price = 3, 5, 300
b_suzuran, b_kasumi, b_price = 2, 3, 190

p_saidai, a_saidai, b_saidai = 0, 0, 0
# グラフを描くために、まず、縦軸 p のリストを用意する
p = []

# 花束セットA が 0 と仮定して開始する
hana_a = 0

while suzuran >= 0 and kasumi >= 0: # 花が残っている限り、繰り返しを続ける
    hana_b = min(int(suzuran / b_suzuran), int(kasumi / b_kasumi)) # 現在の花で作れるセットBの数
    price = a_price * hana_a + b_price * hana_b # 現在の売上合計
    print('A = ', hana_a, ', B = ', hana_b, ', P = ', price)  # 確認用
    p.append(price) # 価格を追加
    if (price > p_saidai): # 売上合計が、それまでより高額であれば
        p_saidai, a_saidai, b_saidai = price, hana_a, hana_b # 最大値となっているところを記録
    hana_a += 1 # 次の繰り返しのためにセットA をひとつ増やす
    suzuran -= a_suzuran # 次の繰り返しのためにセットA をひとつ分のすずらんを使う
    kasumi -= a_kasumi # 次の繰り返しのためにセットA をひとつ分のかすみ草を使う

plt.xlabel("A")
plt.ylabel("price")
plt.plot(p)
print('Result: A = ', a_saidai, ', B = ', b_saidai, ', P = ', p_saidai)

### 確認問題　（確認問題の正答はありません。授業中に提示します）

1. 上記のプログラムの、それぞれの価格や、セットの組み方、仕入れられる分量を変化させてみて、最多の利益となるための販売戦略を考えよ。

2. 上記と同様の構造を持つ問題を考え、それを解いてみよ。例えば、「2つの車種の燃費と汚染物質と積載量」「2つの栄養素のカロリーと脂質とビタミン量」など。

### 参考

線形計画法を解くモジュールがあり、それを利用すると、簡単に正解を得ることができる。ただし、この授業では、アルゴリズムを学ぶことが目的であるため、この部分については説明しない。

【プログラム7602】

In [ ]:
# 線形計画法モジュールによる求解
!pip install pulp
# 線形計画法のモジュールをインポート
from pulp import *

suzuran = 90
kasumi = 140
a_suzuran, a_kasumi, a_price = 3, 5, 300
b_suzuran, b_kasumi, b_price = 2, 3, 190

# オブジェクトを作る

# 最大値問題という設定
problem = LpProblem(name='問題', sense=LpMaximize)

#変化させる変数の宣言
hana_a = LpVariable('hana_a', lowBound=0, cat="Integer")
hana_b = LpVariable('hana_b', lowBound=0, cat="Integer")

# 目的関数 P の定義
problem.objective += a_price * hana_a + b_price * hana_b

# 制約条件
problem += a_suzuran * hana_a + b_suzuran * hana_b <= suzuran
problem += a_kasumi  * hana_a + b_kasumi  * hana_b <= kasumi
problem += hana_a >= 0
problem += hana_b >= 0

#解く
print(problem)
problem.solve()
print("Status : ", LpStatus[problem.status])

# 結果（最大値）
print("P = ", value(problem.objective))

#そのときの変数の値を表示する
for v in problem.variables():
    print(v.name, "=", v.varValue)